In [ ]:
# Для парсинга будет использоваться Selenium и Beautiful Soup
# Данный подход был выбран, так как на сайте есть динамическая подгрузка при скроллинге, соответственно мы не можем использовать только BS
# При этом нет необходимости для запуска паука scrapy, так как не нужно переходить отдельно в каждую карточку товара

# Код запускался локально

In [ ]:
# Сделаем необходимые импорты
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
from selenium.webdriver.common.action_chains import ActionChains

import csv
import re

In [ ]:
START_URL = 'https://www.nbcomputers.ru/catalog/noutbuki/'
html = ''
# инициализация драйвера
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 30)
actions = ActionChains(driver)

driver.get(START_URL)
driver.implicitly_wait(100)
driver.maximize_window()
actions = ActionChains(driver)

In [ ]:
page = 1
def scrollDownToButton():  #функция скроллинга страницы сайта до "конца"
    height = driver.execute_script("return document.body.offsetHeight")   #изменения высота страницы с помощью JS
    y = 1000
    while y <= height:
        sleep(1)   #имитация работы пользователя
        driver.execute_script("window.scrollTo(0, "+str(y)+")")
        y += 1000
    return

In [ ]:
try:
    while True:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.CatalogGrid_inner__s_l_C")))
        scrollDownToButton()
        cards = driver.find_elements(By.CSS_SELECTOR, "div.CatalogCard_main__ZO36s")
        for card in cards:
            html += card.get_attribute("outerHTML")

        buttonNext = driver.find_element(By.CSS_SELECTOR, "li.ant-pagination-next")
        if not buttonNext:
            break

        # Перелистывние страниц и сбор данных с дальнейших страниц
        actions.move_to_element(buttonNext)
        actions.perform()
        buttonNext.click()
        page +=1
except Exception as ex:  #вывод ошибок
    print(ex)

sleep(5)  #имитация работы пользователя
print('Exit url: ', driver.current_url)

# Закрытие страницы
driver.quit()

with open('nbcomputers.html', 'w', encoding='utf-8') as f:
    f.write(html)

In [ ]:
# обработка полученного html файла с помощью bs
file = open('nbcomputers.html', 'r', encoding='utf-8')
html = file.read()

pattern = re.compile(r'\s+')

soup = BeautifulSoup(html, 'lxml')
cards = soup.find_all('div', class_='CatalogCard_main__ZO36s')
data = []

for card in cards:
    code = card.find('p', class_='CatalogHeader_kod__uYN24').get_text()
    title = card.find('a', class_='CatalogHeader_titleLink__4bvZJ')
    link = 'https://www.nbcomputers.ru' + title.get('href')
    name = title.get_text()
    name = name.split('/')[0]
    price = card.find('span', class_='sc-96470d6e-2 bUfXpi').get_text()
    price = pattern.sub(' ', price)

    resolution = ''
    gpu = ''
    ram = ''
    matrix = ''
    cpu = ''
    features = card.find('div', class_='CatalogDescription_description__KxTHX')
    dd_tags_for_features = features.find_all('dd')

    # значения для отбора были взяты на основе вариантов фильтрации на сайте
    for dd_tag in dd_tags_for_features:
        text_in_tag = dd_tag.get_text(strip=True).lower()
        if 'x' in text_in_tag and 'точек' in text_in_tag:
            resolution = text_in_tag
        elif any(gpu_kw in text_in_tag for gpu_kw in ['qualcomm', 'intel arc', 'geforce']):
            gpu = text_in_tag
        elif 'гб' in text_in_tag:
            ram = text_in_tag
        elif 'ips' in text_in_tag or 'tn' in text_in_tag or 'oled' in text_in_tag or 'va' in text_in_tag:
            matrix = text_in_tag
        elif 'intel' in text_in_tag or 'nvidia' in text_in_tag or 'amd' in text_in_tag or 'apple' in text_in_tag:
            cpu = text_in_tag

    data.append({'Код товара': code, 'Ссылка на товар': link, 'Название ноутбука': name, 'Разрешение экрана': resolution,
                 'Дискретная видеокарта': gpu, 'Объем оперативной памяти': ram, 'Тип матрицы': matrix, 'Процессор': cpu, 'Актуальная цена': price})


print(len(data))
if len(data) > 0:
    with open('nbcomputers.csv', 'w', encoding='utf-8') as f:
        fieldnames = data[0].keys()
        dict_writer = csv.DictWriter(f, fieldnames=fieldnames)
        dict_writer.writeheader()
        dict_writer.writerows(data)